In [1]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

Загружаем текст на котором будем обучать

In [2]:
TRAIN_TEXT_FILE_PATH = 'war_and_peace.ru.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

In [3]:
def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'а', 'е', 'и', 'н', 'т', 'с', 'л', 'в', 'р', 'к', 'д', ',', 'м', 'у', 'п', 'я', 'ь', 'г', 'ы', 'з', 'б', 'ч', '.', 'й', 'ж', 'ш', 'х', '–', 'ю', 'e', '\n', 'ц', 'щ', 'Н', 'э', 's', 'n', 'a', 'i', 'r', 'o', 'u', 'П', 't', 'А', 'ф', 'В', 'l', 'О', '?', '!', 'К', 'Б', 'Д', 'm', 'М', 'c', 'd', 'С', '…', 'Р', 'ё', 'p', 'И', ';', 'Т', ':', 'v', '«', '»', 'Я', "'", '[', ']', 'h', 'ъ', 'Г', 'Э', 'Ч', 'Е', 'f', 'q', ')', 'g', 'b', '(', 'I', 'Л', 'У', 'З', 'z', 'j', '0', 'M', '1', 'Ж', 'V', 'B', 'A', 'X', 'Ф', 'Х', 'x', 'Ш', 'L', '8', 'y', 'C', 'J', 'N', 'P', '2', 'S', 'D', '3', 'E', 'k', '5', '6', '4', 'Ц', '7', 'w', 'R', 'T', 'O', 'Q', '9', 'G', 'F', 'H', 'U', '„', '“', 'K', 'Ю', 'W', 'Ь', '`', 'Z', '*', 'Й', 'Щ', '&']


In [4]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [5]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [6]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 25000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)
        print(f"Epoch: {epoch}")

Loss: 3.155982756614685
 ко рото рели не села нель колись пото содо вет то вело на веле пололо сет корого бели селал пото стерене в пото сто воране то на на то и стот и на олило не не кото пото росни то полота сто корариль ол
Epoch: 49
Loss: 2.489899492263794
 не половен в было воменил сен и поворал стороли встовали в пон емела в столим в сказал сет свер полен стостелья оттом стово сказал он вазал сторина в самет волен столения в постолинил стола в нествили
Epoch: 99
Loss: 2.260239419937134
 он в воворил не во сторал подном на сторое предне не и не не вольно в праговорил с не за посторова столь не замение не с воловал в не вого востов не в был с столь с мин на в серин не престово столько 
Epoch: 149
Loss: 2.135374178886414
 не столи одна в предне не головорил к то придиние не доворенно не придиния он продила поломого не принем в простовенно только в постарил и подно своего старил с он не подной польдо ни был не столь пол
Epoch: 199
Loss: 2.0395965147018433
 всего провидить старым и не 

на обучение ушло 1 час 40 минут чистого времении


In [14]:
# model = torch.load('model_v-war_and_peace.pt')
model.eval()
print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=200, 
    start_text='дом был'
    )
)

дом былась к своему руками и с него. Он поднял на своей последний волос на своего голосом и с такой восторженным старого себе не видел старый раз на него, который подошел к нему и не сказала она с другой сто


In [9]:
torch.save(model.state_dict(), 'model_v-war_and_peace.pt')